# Physics Teacher Data Analysis

Let's begin by exploring the physics teacher dataset and examining all available columns.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

: 

In [ ]:
# Load the physics teacher data
# Note: Replace 'physics_data.csv' with the actual filename/path of your data
try:
    df = pd.read_csv('physics_data.csv')
    print("Data loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print("\n" + "="*50)
    print("ALL COLUMNS IN THE DATASET:")
    print("="*50)
    
    # Print all column names
    for i, column in enumerate(df.columns, 1):
        print(f"{i:2d}. {column}")
    
    print(f"\nTotal number of columns: {len(df.columns)}")
    
except FileNotFoundError:
    print("File not found. Please check the filename and path.")
    print("Available files in current directory:")
    import os
    files = [f for f in os.listdir('.') if f.endswith(('.csv', '.xlsx', '.json'))]
    for file in files:
        print(f"  - {file}")